# OpenVINO™ model conversion API

This notebook shows how to convert a model from original framework format to OpenVINO Intermediate Representation (IR).


#### Table of contents:

- [OpenVINO IR format](#OpenVINO-IR-format)
- [IR preparation with Python conversion API and Model Optimizer command-line tool](#IR-preparation-with-Python-conversion-API-and-Model-Optimizer-command-line-tool)
- [Fetching example models](#Fetching-example-models)
- [Basic conversion](#Basic-conversion)
- [Model conversion parameters](#Model-conversion-parameters)
    - [Setting Input Shapes](#Setting-Input-Shapes)
    - [Cutting Off Parts of a Model](#Cutting-Off-Parts-of-a-Model)
    - [Embedding Preprocessing Computation](#Embedding-Preprocessing-Computation)
        - [Specifying Layout](#Specifying-Layout)
        - [Changing Model Layout](#Changing-Model-Layout)
        - [Specifying Mean and Scale Values](#Specifying-Mean-and-Scale-Values)
        - [Reversing Input Channels](#Reversing-Input-Channels)
    - [Compressing a Model to FP16](#Compressing-a-Model-to-FP16)
- [Convert Models Represented as Python Objects](#Convert-Models-Represented-as-Python-Objects)



In [1]:
# Required imports. Please execute this cell first.
%pip install -q --extra-index-url https://download.pytorch.org/whl/cpu \
"openvino-dev>=2024.0.0" "requests" "tqdm" "transformers[onnx]>=4.21.1" "torch>=2.1" "torchvision"

## OpenVINO IR format
[back to top ⬆️](#Table-of-contents:)

OpenVINO [Intermediate Representation (IR)](https://docs.openvino.ai/2024/documentation/openvino-ir-format.html) is the proprietary model format of OpenVINO. It is produced after converting a model with model conversion API. Model conversion API translates the frequently used deep learning operations to their respective similar representation in OpenVINO and tunes them with the associated weights and biases from the trained model. The resulting IR contains two files: an `.xml` file, containing information about network topology, and a `.bin` file, containing the weights and biases binary data.

## IR preparation with Python conversion API and Model Optimizer command-line tool
[back to top ⬆️](#Table-of-contents:)

There are two ways to convert a model from the original framework format to OpenVINO IR: Python conversion API and Model Optimizer command-line tool. You can choose one of them based on whichever is most convenient for you. There should not be any differences in the results of model conversion if the same set of parameters is used. For more details, refer to [Model Preparation](https://docs.openvino.ai/2024/openvino-workflow/model-preparation.html) documentation.

In [2]:
# Model Optimizer CLI tool parameters description

! mo --help

usage: main.py [options]

optional arguments:
  -h, --help            show this help message and exit
  --framework FRAMEWORK
                        Name of the framework used to train the input model.

Framework-agnostic parameters:
  --model_name MODEL_NAME, -n MODEL_NAME
                        Model_name parameter passed to the final create_ir
                        transform. This parameter is used to name a network in
                        a generated IR and output .xml/.bin files.
  --output_dir OUTPUT_DIR, -o OUTPUT_DIR
                        Directory that stores the generated IR. By default, it
                        is the directory from where the Model Conversion is
                        launched.
  --freeze_placeholder_with_value FREEZE_PLACEHOLDER_WITH_VALUE
                        Replaces input layer with constant node with provided
                        value, for example: "node_name->True". It will be
                        DEPRECATED in future releases. Us

In [3]:
# Python conversion API parameters description
from openvino.tools import mo


mo.convert_model(help=True)

Optional parameters:
  --help 
			Print available parameters.
  --framework 
			Name of the framework used to train the input model.

Framework-agnostic parameters:
  --input_model 
			Model object in original framework (PyTorch, Tensorflow) or path to
			model file.
			Tensorflow*: a file with a pre-trained model (binary or text .pb file
			after freezing).
			Caffe*: a model proto file with model weights
			
			Supported formats of input model:
			
			PaddlePaddle
			paddle.hapi.model.Model
			paddle.fluid.dygraph.layers.Layer
			paddle.fluid.executor.Executor
			
			PyTorch
			torch.nn.Module
			torch.jit.ScriptModule
			torch.jit.ScriptFunction
			
			TF
			tf.compat.v1.Graph
			tf.compat.v1.GraphDef
			tf.compat.v1.wrap_function
			tf.compat.v1.session
			
			TF2 / Keras
			tf.keras.Model
			tf.keras.layers.Layer
			tf.function
			tf.Module
			tf.train.checkpoint
  --input 
			Input can be set by passing a list of InputCutInfo objects or by a list
			of tuples. Each tuple can cont

## Fetching example models
[back to top ⬆️](#Table-of-contents:)

This notebook uses two models for conversion examples:

* [Distilbert](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) NLP model from Hugging Face
* [Resnet50](https://pytorch.org/vision/stable/models/generated/torchvision.models.resnet50.html#torchvision.models.ResNet50_Weights) CV classification model from torchvision

In [4]:
from pathlib import Path

# create a directory for models files
MODEL_DIRECTORY_PATH = Path("model")
MODEL_DIRECTORY_PATH.mkdir(exist_ok=True)

Fetch [distilbert](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) NLP model from Hugging Face and export it in ONNX format:

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers.onnx import export, FeaturesManager


ONNX_NLP_MODEL_PATH = MODEL_DIRECTORY_PATH / "distilbert.onnx"

# download model
hf_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# get model onnx config function for output feature format sequence-classification
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(hf_model, feature="sequence-classification")
# fill onnx config based on pytorch model config
onnx_config = model_onnx_config(hf_model.config)

# export to onnx format
export(
    preprocessor=tokenizer,
    model=hf_model,
    config=onnx_config,
    opset=onnx_config.default_onnx_opset,
    output=ONNX_NLP_MODEL_PATH,
)

2023-10-30 09:15:39.568630: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-30 09:15:39.665054: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 09:15:41.296271: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ea/work/ov_venv/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11080). Please update your GPU driver by downloading and installing a new versi

(['input_ids', 'attention_mask'], ['logits'])

Fetch [Resnet50](https://pytorch.org/vision/stable/models/generated/torchvision.models.resnet50.html#torchvision.models.ResNet50_Weights) CV classification model from torchvision:

In [6]:
from torchvision.models import resnet50, ResNet50_Weights


# create model object
pytorch_model = resnet50(weights=ResNet50_Weights.DEFAULT)
# switch model from training to inference mode
pytorch_model.eval()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /home/ea/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:06<00:00, 14.9MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Convert PyTorch model to ONNX format:

In [7]:
import torch
import warnings


ONNX_CV_MODEL_PATH = MODEL_DIRECTORY_PATH / "resnet.onnx"

if ONNX_CV_MODEL_PATH.exists():
    print(f"ONNX model {ONNX_CV_MODEL_PATH} already exists.")
else:
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        torch.onnx.export(model=pytorch_model, args=torch.randn(1, 3, 780, 520), f=ONNX_CV_MODEL_PATH)
    print(f"ONNX model exported to {ONNX_CV_MODEL_PATH}")

ONNX model exported to model/resnet.onnx


## Basic conversion
[back to top ⬆️](#Table-of-contents:)

To convert a model to OpenVINO IR, use the following command:

In [8]:
# Model Optimizer CLI

! mo --input_model model/distilbert.onnx --output_dir model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.bin


In [9]:
# Python conversion API
from openvino.tools import mo

# mo.convert_model returns an openvino.runtime.Model object
ov_model = mo.convert_model(ONNX_NLP_MODEL_PATH)

# then model can be serialized to *.xml & *.bin files
from openvino.runtime import serialize

serialize(ov_model, xml_path=MODEL_DIRECTORY_PATH / "distilbert.xml")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Model conversion parameters
[back to top ⬆️](#Table-of-contents:)

Both Python conversion API and Model Optimizer command-line tool provide the following capabilities:
* overriding original input shapes for model conversion with `input` and `input_shape` parameters. [Setting Input Shapes guide](https://docs.openvino.ai/2024/openvino-workflow/model-preparation/setting-input-shapes.html).
* cutting off unwanted parts of a model (such as unsupported operations and training sub-graphs) using the `input` and `output` parameters to define new inputs and outputs of the converted model. [Cutting Off Parts of a Model guide](https://docs.openvino.ai/2023.3/openvino_docs_MO_DG_prepare_model_convert_model_Cutting_Model.html).
* inserting additional input pre-processing sub-graphs into the converted model by using the `mean_values`, `scales_values`, `layout`, and other parameters. [Embedding Preprocessing Computation article](https://docs.openvino.ai/2023.3/openvino_docs_MO_DG_Additional_Optimization_Use_Cases.html).
* compressing the model weights (for example, weights for convolutions and matrix multiplications) to FP16 data type using `compress_to_fp16` compression parameter. [Compression of a Model to FP16 guide](https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html).

If the out-of-the-box conversion (only the `input_model` parameter is specified) is not successful, it may be required to use the parameters mentioned above to override input shapes and cut the model.

### Setting Input Shapes
[back to top ⬆️](#Table-of-contents:)

Model conversion is supported for models with dynamic input shapes that contain undefined dimensions. However, if the shape of data is not going to change from one inference request to another, it is recommended to set up static shapes (when all dimensions are fully defined) for the inputs. Doing it at this stage, instead of during inference in runtime, can be beneficial in terms of performance and memory consumption. To set up static shapes, model conversion API provides the `input` and `input_shape` parameters.

For more information refer to [Setting Input Shapes guide](https://docs.openvino.ai/2024/openvino-workflow/model-preparation/setting-input-shapes.html).

In [10]:
# Model Optimizer CLI

! mo --input_model model/distilbert.onnx --input input_ids,attention_mask --input_shape [1,128],[1,128] --output_dir model

# alternatively
! mo --input_model model/distilbert.onnx --input input_ids[1,128],attention_mask[1,128] --output_dir model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.bin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.bin


In [11]:
# Python conversion API
from openvino.tools import mo


ov_model = mo.convert_model(
    ONNX_NLP_MODEL_PATH,
    input=["input_ids", "attention_mask"],
    input_shape=[[1, 128], [1, 128]],
)

# alternatively specify input shapes, using the input parameter
ov_model = mo.convert_model(ONNX_NLP_MODEL_PATH, input=[("input_ids", [1, 128]), ("attention_mask", [1, 128])])

The input_shape parameter allows overriding original input shapes to ones compatible with a given model. Dynamic shapes, i.e. with dynamic dimensions, can be replaced in the original model with static shapes for the converted model, and vice versa. The dynamic dimension can be marked in the model conversion API parameter as `-1` or `?`. For example, launch model conversion for the ONNX Bert model and specify a dynamic sequence length dimension for inputs:

In [12]:
# Model Optimizer CLI

! mo --input_model model/distilbert.onnx --input input_ids,attention_mask --input_shape [1,-1],[1,-1] --output_dir model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.bin


In [13]:
# Python conversion API
from openvino.tools import mo


ov_model = mo.convert_model(
    ONNX_NLP_MODEL_PATH,
    input=["input_ids", "attention_mask"],
    input_shape=[[1, -1], [1, -1]],
)

To optimize memory consumption for models with undefined dimensions in runtime, model conversion API provides the capability to define boundaries of dimensions. The boundaries of undefined dimensions can be specified with ellipsis. For example, launch model conversion for the ONNX Bert model and specify a boundary for the sequence length dimension:

In [14]:
# Model Optimizer CLI

! mo --input_model model/distilbert.onnx --input input_ids,attention_mask --input_shape [1,10..128],[1,10..128] --output_dir model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.bin


In [15]:
# Python conversion API
from openvino.tools import mo


ov_model = mo.convert_model(
    ONNX_NLP_MODEL_PATH,
    input=["input_ids", "attention_mask"],
    input_shape=[[1, "10..128"], [1, "10..128"]],
)

### Cutting Off Parts of a Model
[back to top ⬆️](#Table-of-contents:)

The following examples show when model cutting is useful or even required:

* A model has pre- or post-processing parts that cannot be translated to existing OpenVINO operations.
* A model has a training part that is convenient to be kept in the model but not used during inference.
* A model is too complex to be converted at once because it contains many unsupported operations that cannot be easily implemented as custom layers.
* A problem occurs with model conversion or inference in OpenVINO Runtime. To identify the issue, limit the conversion scope by an iterative search for problematic areas in the model.
* A single custom layer or a combination of custom layers is isolated for debugging purposes.

For a more detailed description, refer to the [Cutting Off Parts of a Model guide](https://docs.openvino.ai/2023.3/openvino_docs_MO_DG_prepare_model_convert_model_Cutting_Model.html).

In [16]:
# Model Optimizer CLI

# cut at the end
! mo --input_model model/distilbert.onnx --output /classifier/Gemm --output_dir model


# cut from the beginning
! mo --input_model model/distilbert.onnx --input /distilbert/embeddings/LayerNorm/Add_1,attention_mask --output_dir model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.bin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/distilbert.bin


In [17]:
# Python conversion API
from openvino.tools import mo


# cut at the end
ov_model = mo.convert_model(ONNX_NLP_MODEL_PATH, output="/classifier/Gemm")

# cut from the beginning
ov_model = mo.convert_model(
    ONNX_NLP_MODEL_PATH,
    input=["/distilbert/embeddings/LayerNorm/Add_1", "attention_mask"],
)

### Embedding Preprocessing Computation
[back to top ⬆️](#Table-of-contents:)

Input data for inference can be different from the training dataset and requires additional preprocessing before inference. To accelerate the whole pipeline, including preprocessing and inference, model conversion API provides special parameters such as `mean_values`, `scale_values`, `reverse_input_channels`, and `layout`. Based on these parameters, model conversion API generates OpenVINO IR with additionally inserted sub-graphs to perform the defined preprocessing. This preprocessing block can perform mean-scale normalization of input data, reverting data along channel dimension, and changing the data layout. For more information on preprocessing, refer to the [Embedding Preprocessing Computation article](https://docs.openvino.ai/2023.3/openvino_docs_MO_DG_Additional_Optimization_Use_Cases.html).

#### Specifying Layout
[back to top ⬆️](#Table-of-contents:)

Layout defines the meaning of dimensions in a shape and can be specified for both inputs and outputs. Some preprocessing requires to set input layouts, for example, setting a batch, applying mean or scales, and reversing input channels (BGR<->RGB). For the layout syntax, check the [Layout API overview](https://docs.openvino.ai/2024/openvino-workflow/running-inference/optimize-inference/optimize-preprocessing/layout-api-overview.html). To specify the layout, you can use the layout option followed by the layout value.

The following command specifies the `NCHW` layout for a Pytorch Resnet50 model that was exported to the ONNX format:

In [18]:
# Model Optimizer CLI

! mo --input_model model/resnet.onnx --layout nchw --output_dir model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.bin


In [19]:
# Python conversion API
from openvino.tools import mo


ov_model = mo.convert_model(ONNX_CV_MODEL_PATH, layout="nchw")

#### Changing Model Layout
[back to top ⬆️](#Table-of-contents:)

Changing the model layout may be necessary if it differs from the one presented by input data. Use either `layout` or `source_layout` with `target_layout` to change the layout.

In [20]:
# Model Optimizer CLI

! mo --input_model model/resnet.onnx --layout "nchw->nhwc" --output_dir model

# alternatively use source_layout and target_layout parameters
! mo --input_model model/resnet.onnx --source_layout nchw --target_layout nhwc --output_dir model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.bin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.bin


In [21]:
# Python conversion API
from openvino.tools import mo


ov_model = mo.convert_model(ONNX_CV_MODEL_PATH, layout="nchw->nhwc")

# alternatively use source_layout and target_layout parameters
ov_model = mo.convert_model(ONNX_CV_MODEL_PATH, source_layout="nchw", target_layout="nhwc")

#### Specifying Mean and Scale Values
[back to top ⬆️](#Table-of-contents:)

Model conversion API has the following parameters to specify the values: `mean_values`, `scale_values`, `scale`. Using these parameters, model conversion API embeds the corresponding preprocessing block for mean-value normalization of the input data and optimizes this block so that the preprocessing takes negligible time for inference.

In [22]:
# Model Optimizer CLI

! mo --input_model model/resnet.onnx --mean_values [123,117,104] --scale 255 --output_dir model

! mo --input_model model/resnet.onnx --mean_values [123,117,104] --scale_values [255,255,255] --output_dir model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.bin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.bin


In [23]:
# Python conversion API
from openvino.tools import mo


ov_model = mo.convert_model(ONNX_CV_MODEL_PATH, mean_values=[123, 117, 104], scale=255)

ov_model = mo.convert_model(ONNX_CV_MODEL_PATH, mean_values=[123, 117, 104], scale_values=[255, 255, 255])

#### Reversing Input Channels
[back to top ⬆️](#Table-of-contents:)

Sometimes, input images for your application can be of the `RGB` (or `BGR`) format, and the model is trained on images of the `BGR` (or `RGB`) format, which is in the opposite order of color channels. In this case, it is important to preprocess the input images by reverting the color channels before inference.

In [24]:
# Model Optimizer CLI

! mo --input_model model/resnet.onnx --reverse_input_channels --output_dir model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.bin


In [25]:
# Python conversion API
from openvino.tools import mo


ov_model = mo.convert_model(ONNX_CV_MODEL_PATH, reverse_input_channels=True)

### Compressing a Model to FP16
[back to top ⬆️](#Table-of-contents:)

Optionally all relevant floating-point weights can be compressed to FP16 data type during the model conversion, creating a compressed FP16 model. This smaller model occupies about half of the original space in the file system. While the compression may introduce a drop in accuracy, for most models, this decrease is negligible.

In [26]:
# Model Optimizer CLI

! mo --input_model model/resnet.onnx --compress_to_fp16=True --output_dir model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2024/openvino-workflow/model-preparation/conversion-parameters.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.3/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.xml
[ SUCCESS ] BIN file: /home/ea/work/openvino_notebooks/notebooks/convert-to-openvino/model/resnet.bin


In [27]:
# Python conversion API
from openvino.tools import mo


ov_model = mo.convert_model(ONNX_CV_MODEL_PATH, compress_to_fp16=True)

## Convert Models Represented as Python Objects
[back to top ⬆️](#Table-of-contents:)

Python conversion API can pass Python model objects, such as a Pytorch model or TensorFlow Keras model directly, without saving them into files and without leaving the training environment (Jupyter Notebook or training scripts).

In [28]:
# Python conversion API
from openvino.tools import mo


ov_model = mo.convert_model(pytorch_model)

`convert_model()` accepts all parameters available in the MO command-line tool. Parameters can be specified by Python classes or string analogs, similar to the command-line tool.

In [29]:
# Python conversion API
from openvino.tools import mo


ov_model = mo.convert_model(
    pytorch_model,
    input_shape=[1, 3, 100, 100],
    mean_values=[127, 127, 127],
    layout="nchw",
)

ov_model = mo.convert_model(pytorch_model, source_layout="nchw", target_layout="nhwc")

ov_model = mo.convert_model(pytorch_model, compress_to_fp16=True, reverse_input_channels=True)